# **A* Search Algorithm**

Let's say that you have to get through an enormous maze.

\\

![Figure 1](https://s3.amazonaws.com/stackabuse/media/basic-ai-concepts-a-search-algorithm-1.png)


To make things significantly easier and less time consuming, we'll boil the maze down to a search problem, and come up with a solution that can be applied to any additional maze that we may encounter - as long as it follows the same rules/structure.


Any time we want to convert any kind of problem into a search problem, we have to define six things:

1. A set of all states we might end up in
2. The start and finish state
3. A finish check (a way to check if we're at the finished state)
4. A set of possible actions (in this case, different directions of movement)
5. A traversal function (a function that will tell us where we'll end up if we go a certain direction)
6. A set of movement costs from state-to-state (which correspond to edges in the graph)

Now that we have a finished graph, we can discuss algorithms for finding a path from state A to state B. In simple cases (like this one), where the generated graph consists of a small number of nodes and edges, BFS, DFS and Dijkstra would suffice.

\\

However, in a real-life scenario, because we are dealing with problems of enormous combinatorial complexity, we know we will have to deal with an enormous amount of nodes and edges. For example, there are many states a Rubik's cube can be in, which is why solving it is so difficult. Therefore, we have to use an algorithm that is, in a sense, guided. That is where an informed search algorithm arises, A*.


# Basic Concepts of A*

A* is based on using heuristic methods to achieve optimality and completeness, and is a variant of the best-first algorithm.

When a search algorithm has the property of optimality, it means it is guaranteed to find the best possible solution, in our case the shortest path to the finish state. When a search algorithm has the property of completeness, it means that if a solution to a given problem exists, the algorithm is guaranteed to find it.

Each time A* enters a state, it calculates the cost, f(n) (n being the neighboring node), to travel to all of the neighboring nodes, and then enters the node with the lowest value of f(n).

These values are calculated with the following formula:

                                                               f(n)=g(n)+h(n)

g(n) being the value of the shortest path from the start node to node n, and h(n) being a heuristic approximation of the node's value.

#Admissibility and Consistency

A given heuristic function h(n) is admissible if it never overestimates the real distance between n and the goal node.

Therefore, for every node n the following formula applies:

                                                                h(n) ≤ h*(n)


h*(n) being the real distance between n and the goal node. However, if the function does overestimate the real distance, but never by more than d, we can safely say that the solution that the function produces is of accuracy d (i.e. it doesn't overestimate the shortest path from start to finish by more than d).

A given heuristic function h(n) is consistent if the estimate is always less than or equal to the estimated distance between the goal n and any given neighbor, plus the estimated cost of reaching that neighbor:

                                                           c(n,m) + h(m) ≥ h(n)


c(n,m) being the distance between nodes n and m. Additionally, if h(n) is consistent, then we know the optimal path to any node that has been already inspected. This means that this function is optimal.

**Theorem:** If a heuristic function is consistent, then it is also admissible.

#Proof by Complete Induction

The induction parameter N will be the number of nodes between node n and the finish node s on the shortest path between the two.

**Base:** N=0

If there are no nodes between n and s, and because we know that h(n) is consistent, the following equation is valid:

                                                           c(n,s) + h(s) ≥ h(n)


Knowing  h*(n)=c(n,s) and h(s)=0 we can safely deduce that:

                                                              h*(n) ≥ h(n)
*Induction hypothesis:* N < k

We hypothesize that the given rule is true for every N < k.

*Induction step:*

In the case of N = k nodes on the shortest path from n to s, we inspect the first successor (node m) of the finish node n. Because we know that there is a path from m to n, and we know this path contains k-1 nodes, the following equation is valid:

                                                h*(n) = c(n,m) + h*(m) ≥ c(n,m) + h(m) ≥h(n)


#Implementation

In [2]:
from collections import deque

class Graph:
    # example of adjacency list (or rather map)
    # adjacency_list = {
    # 'A': [('B', 1), ('C', 3), ('D', 7)],
    # 'B': [('D', 5)],
    # 'C': [('D', 12)]
    # }

    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    # heuristic function with equal values for all nodes
    def h(self, n):
        H = {
            'A': 1,
            'B': 1,
            'C': 1,
            'D': 1
        }

        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        # open_list is a list of nodes which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = set([start_node])
        closed_list = set([])

        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}

        g[start_node] = 0

        # parents contains an adjacency map of all nodes
        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None

            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v;

            if n == None:
                print('Path does not exist!')
                return None

            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == stop_node:
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None


adjacency_list = {
    'A': [('B', 1), ('C', 3), ('D', 7)],
    'B': [('D', 5)],
    'C': [('D', 12)]
}
graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('A', 'D')




Path found: ['A', 'B', 'D']


['A', 'B', 'D']